https://docs.ragas.io/en/stable/getstarted/rag_eval/

In [11]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [12]:
import numpy as np
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import os

class RAG:
    def __init__(self):
        self.llm = ChatOpenAI(
                        model=os.getenv("GENERATOR_MODEL"),
                        api_key=os.getenv("GENERATOR_API_KEY"),
                        base_url=os.getenv("GENERATOR_BASE_URL")
                    )
        self.embeddings = OpenAIEmbeddings(
                        model=os.getenv("EMBEDDER_MODEL"),
                        api_key=os.getenv("EMBEDDER_API_KEY"),
                        base_url=os.getenv("EMBEDDER_BASE_URL")
                    )
        self.doc_embeddings = None
        self.docs = None

    def load_documents(self, documents):
        """Load documents and compute their embeddings."""
        self.docs = documents
        self.doc_embeddings = self.embeddings.embed_documents(documents)

    def get_most_relevant_docs(self, query):
        """Find the most relevant document for a given query."""
        if not self.docs or not self.doc_embeddings:
            raise ValueError("Documents and their embeddings are not loaded.")

        query_embedding = self.embeddings.embed_query(query)
        similarities = [
            np.dot(query_embedding, doc_emb)
            / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
            for doc_emb in self.doc_embeddings
        ]
        most_relevant_doc_index = np.argmax(similarities)
        return [self.docs[most_relevant_doc_index]]

    def generate_answer(self, query, relevant_doc):
        """Generate an answer for a given query based on the most relevant document."""
        prompt = f"question: {query}\n\nDocuments: {relevant_doc}"
        messages = [
            ("system", "You are a helpful assistant that answers questions based on given documents only."),
            ("user", prompt),
        ]
        ai_msg = self.llm.invoke(messages)
        return ai_msg.content

In [13]:
from langchain_community.document_loaders import DirectoryLoader

path = "docs/"
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"), 
    ("###", "Header 3"),
    ("####", "Header 4"),
    ("#####", "Header 5"),
    ("######", "Header 6"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False,  # Keep headers in content for context
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    add_start_index=True,
)
print("✅ Text Chunkers initialized.")

print("\n🔄 Chunking documents...")
md_split_docs = []
for doc in docs:
    md_header_splits = markdown_splitter.split_text(doc.page_content)
    md_header_splits = text_splitter.split_documents(md_header_splits)
    for split_chunk in md_header_splits:
        md_split_docs.append(split_chunk.page_content)
print(f"👍 Documents split into {len(md_split_docs)} chunks.")
docs = md_split_docs

✅ Text Chunkers initialized.

🔄 Chunking documents...
👍 Documents split into 46 chunks.


In [15]:
print(docs[0])

《神奇之门: 奇门遁甲大解谜》  
张志春著  
上 编  
易学思维中的科学性精华  
第一章  
易学的科学价值主要在思维科学  
1. 易学的历史发展  
起源与演变：从伏羲画八卦开始，历经夏《连山》、商《归藏》、周《周易》的成型，到孔子作"十翼"（易传），汉代列为群经之首，形成完整的易学体系。  
流派分化：清代《四库全书》概括易学为"两派六宗"------象数派（含象数、机祥、图书）与义理派（含儒理、考史），两者长期争论并存，涵盖哲学、天文、军事等广泛领域。  
2. 易学的现代研究  
全球影响：近代以来，易学从中国走向世界，引发跨学科研究热潮，包括社会科学、自然科学的多元视角。  
科学评价：中外学者（如邓拓、荣格、张协和）高度评价《周易》的智慧价值，认为其宇宙观与规律性研究对现代科学（如量子物理）有启发，甚至与诺贝尔奖成果相关。  
3. 易学的科学价值争议  
观点分歧：  
部分人认为价值仅在"易理"（哲学），否定"象数/数术"的科学性；  
另一派认为"象数"是易理的应用体现，两者不可割裂；  
折中观点主张"学"（理论）与"术"（应用）结合，古为今用。  
作者立场：支持第三种观点，强调象、数、理、占是统一整体，易学价值需从理论和应用两方面综合研究。  
4. 易学的科学定位  
思维科学为核心：作者引用钱学森的现代科学体系分类，提出易学应归入思维科学，因其提供了独特的认知模式（如辩证、象数思维），对自然科学、社会科学及人生实践均有指导意义。  
实践价值：易学不仅包含哲学思想（基础科学层次），也通过数术技术（工程技术层次）联系实际，促进物质与精神文明发展。  
5. 结论  
易学作为中华文化的智慧总汇，其核心价值在于思维科学领域，兼具理论深度与实践意义，需以开放态度继承发展，服务于现代科学与文明建设。  
第二章  
人脑思维方式与易学的产生  
1. 人类思维方式的分类  
现代思维科学将人类思维方式分为三类：  
逻辑思维（理性思维）：代表学科如数学、物理、化学。  
形象思维（感性思维）：代表学科如文学、艺术、音乐。  
灵感思维（感应思维/直觉思维）：代表学科如易学、佛学、心理学、气功。  
人类思维发展的规律：  
婴幼儿以感性思维为主，随着年龄增长，逻辑思维逐渐增强。


In [17]:
# Initialize RAG instance
rag = RAG()

# Load documents
rag.load_documents(docs)

# Query and retrieve the most relevant document
query = "中国古代智慧，特别是易学，与现代科学思想有何关联？以具体的例子来证明这种关联。"
relevant_doc = rag.get_most_relevant_docs(query)

# Generate an answer
answer = rag.generate_answer(query, relevant_doc)

print(f"Query: {query}")
print(f"Relevant Document: {relevant_doc}")
print(f"Answer: {answer}")

Query: 中国古代智慧，特别是易学，与现代科学思想有何关联？以具体的例子来证明这种关联。
Relevant Document: ['《神奇之门: 奇门遁甲大解谜》  \n张志春著  \n上 编  \n易学思维中的科学性精华  \n第一章  \n易学的科学价值主要在思维科学  \n1. 易学的历史发展  \n起源与演变：从伏羲画八卦开始，历经夏《连山》、商《归藏》、周《周易》的成型，到孔子作"十翼"（易传），汉代列为群经之首，形成完整的易学体系。  \n流派分化：清代《四库全书》概括易学为"两派六宗"------象数派（含象数、机祥、图书）与义理派（含儒理、考史），两者长期争论并存，涵盖哲学、天文、军事等广泛领域。  \n2. 易学的现代研究  \n全球影响：近代以来，易学从中国走向世界，引发跨学科研究热潮，包括社会科学、自然科学的多元视角。  \n科学评价：中外学者（如邓拓、荣格、张协和）高度评价《周易》的智慧价值，认为其宇宙观与规律性研究对现代科学（如量子物理）有启发，甚至与诺贝尔奖成果相关。  \n3. 易学的科学价值争议  \n观点分歧：  \n部分人认为价值仅在"易理"（哲学），否定"象数/数术"的科学性；  \n另一派认为"象数"是易理的应用体现，两者不可割裂；  \n折中观点主张"学"（理论）与"术"（应用）结合，古为今用。  \n作者立场：支持第三种观点，强调象、数、理、占是统一整体，易学价值需从理论和应用两方面综合研究。  \n4. 易学的科学定位  \n思维科学为核心：作者引用钱学森的现代科学体系分类，提出易学应归入思维科学，因其提供了独特的认知模式（如辩证、象数思维），对自然科学、社会科学及人生实践均有指导意义。  \n实践价值：易学不仅包含哲学思想（基础科学层次），也通过数术技术（工程技术层次）联系实际，促进物质与精神文明发展。  \n5. 结论  \n易学作为中华文化的智慧总汇，其核心价值在于思维科学领域，兼具理论深度与实践意义，需以开放态度继承发展，服务于现代科学与文明建设。  \n第二章  \n人脑思维方式与易学的产生  \n1. 人类思维方式的分类  \n现代思维科学将人类思维方式分为三类：  \n逻辑思维（理性思维）：代表学科如数学、物理、化学。  \n形象思维（感性思维）：代表学科如文学、艺术、音乐。  \n灵感思维（感应思维/

In [31]:
import pandas as pd
from ragas.dataset_schema import EvaluationDataset

df = pd.read_parquet("data/ragas_openai_testset.parquet")
df["retrieved_contexts"] = df["reference_contexts"]
df["response"] = df["reference"]
data_list = df.to_dict(orient="records")
evaluation_dataset = EvaluationDataset.from_list(data_list)
evaluation_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'reference_contexts', 'response', 'reference'], len=10)

In [32]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper


evaluator_llm = LangchainLLMWrapper(rag.llm)
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

result = evaluate(dataset=evaluation_dataset,
                  metrics=[
                    LLMContextRecall(), 
                    Faithfulness(), 
                    FactualCorrectness()],
                  llm=evaluator_llm
                )
result

Evaluating: 100%|██████████| 30/30 [03:58<00:00,  7.96s/it]


{'context_recall': 1.0000, 'faithfulness': 0.9444, 'factual_correctness(mode=f1)': 1.0000}